### Análise de Dados com Python e SQL

Este estudo visa explorar os dados fornecidos pela plataforma Kaggle referentes a oportunidades de emprego no estado da Califórnia, nos Estados Unidos. O principal objetivo desta análise é empregar técnicas para transformar e examinar os dados disponíveis, visando extrair insights e informações relevantes.

Para realizar essa análise, faremos uso das linguagens Python e SQL, sendo esta última empregada com a sintaxe específica do SQLite.

[Link para os dados](https://www.kaggle.com/datasets/datasciencedonut/california-state-jobs/)


In [1]:
# Bibliotecas Importadas
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
# Dicionario dos dados
dicionario = pd.read_csv('colunas.csv', sep=';', encoding='ISO-8859-1')
dicionario

,Coluna,Descriçao
0,Job_Listing,"Título do cargo, como seria mostrado no regist..."
1,Working_Title,"Para categorias mais amplas, o título provisór..."
2,Job_Control,Número exclusivo atribuído ao cargo
3,Salary_Range,A remuneração mínima e máxima para o cargo.
4,Work_Type/Schedule,Tempo de compromisso e tempo de trabalho
5,Department,O departamento ou agência da Califórnia que of...
6,Location,Onde o cargo/departamento está localizado
7,Publish_Date,Data em que a lista de empregos foi publicada
8,Filing_Deadline,Data em que os pedidos de emprego têm de ser a...
9,URLs,A URL da listagem de vagas


In [3]:
# Importando os dados e conectando com um banco de dados do SQLite
df = pd.read_csv('CalCareersData.csv', index_col = 1)
conn = sql.connect('CalCareers.db')

#### Limpeza dos dados
Ao iniciar uma análise exploratória, a etapa inicial crucial é a limpeza da base de dados que será analisada. Embora seja possível realizar essa limpeza através do SQL, especialmente quando há necessidade de modificar a estrutura dos dados, no nosso caso específico, optaremos por utilizar o Python. 

In [4]:
query = '''
SELECT *
FROm tabela
LIMIT 5
'''
df_read = pd.read_sql(query, con = conn)
df_read

,Working_Title,Job_Listing,Job Control,Salary_Range,Work_Type/Schedule,Department,Location,Publish_Date,Filing_Deadline,URLs,Salary_Min,Salary_Max
0,ACCOUNTANT TRAINEE,ACCOUNTANT TRAINEE,403508,$4065.00 - $4847.00,Permanent Fulltime,State Controller's Office,Sacramento County,2023-11-17T00:00:00,2023-11-27T00:00:00,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,4065.00,4847.00
1,Payroll Disbursement Staff,ACCOUNTING OFFICER (SPECIALIST),397289,$4819.00 - $6032.00,Permanent Fulltime,Department of Water Resources,Sacramento County,2023-11-17T00:00:00,2023-11-27T00:00:00,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,4819.00,6032.00
2,INFORMATION TECHNOLOGY MANAGER I,INFORMATION TECHNOLOGY MANAGER I,403219,$8591.00 - $11512.00,Permanent Fulltime,Department of Water Resources,Sacramento County,2023-11-17T00:00:00,2023-12-18T00:00:00,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,8591.00,11512.00
3,CSC Support,INFORMATION TECHNOLOGY SPECIALIST I,403517,$5960.00 - $9643.00,Permanent Fulltime,State Controller's Office,Sacramento County,2023-11-17T00:00:00,None,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,5960.00,9643.00
4,Property Appraiser,JUNIOR PROPERTY APPRAISER,403535,$3534.00 - $4216.00,Permanent Fulltime,Board of Equalization,Sacramento County,2023-11-17T00:00:00,2023-12-11T00:00:00,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,3534.00,4216.00


In [5]:
# Para conhecer melhor as colunas iremos avalia-las pela clausa info().
# Essas informações podem ser facilmente obtidas diretamente do banco de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9827 entries, ACCOUNTANT TRAINEE to FOOD SERVICE SUPERVISOR I
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Job_Listing         9827 non-null   object
 1   Job Control         9827 non-null   int64 
 2   Salary_Range        9827 non-null   object
 3   Work_Type/Schedule  9827 non-null   object
 4   Department          9827 non-null   object
 5   Location            9827 non-null   object
 6   Publish_Date        9827 non-null   object
 7   Filing_Deadline     9827 non-null   object
 8   URLs                9827 non-null   object
dtypes: int64(1), object(8)
memory usage: 767.7+ KB


In [6]:
# Identificamos que as colunas "Publish_Date" esta categorizada como 'object', vamos transforma-la em 'datetime'

df['Publish_Date'] = pd.to_datetime(df['Publish_Date'], format='%m/%d/%Y')

# Existem algumas linhas com o valor "Until Filled" na coluna "Filing_deadline",
# o que significa que a vaga ficará com o status 'aberta' até que seja preenchida. Para corrigimos o tipo da coluna vamos substituir o valor "Until Filled" por "NaT" que significa "Not a Time"

df['Filing_Deadline'] = df['Filing_Deadline'].replace('Until Filled', np.nan)
df['Filing_Deadline'] = pd.to_datetime(df['Filing_Deadline'], format='%m/%d/%Y')


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9827 entries, ACCOUNTANT TRAINEE to FOOD SERVICE SUPERVISOR I
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Job_Listing         9827 non-null   object        
 1   Job Control         9827 non-null   int64         
 2   Salary_Range        9827 non-null   object        
 3   Work_Type/Schedule  9827 non-null   object        
 4   Department          9827 non-null   object        
 5   Location            9827 non-null   object        
 6   Publish_Date        9827 non-null   datetime64[ns]
 7   Filing_Deadline     8130 non-null   datetime64[ns]
 8   URLs                9827 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 767.7+ KB


In [8]:
# Iremos separa os valores da coluna "Salary_range" em duas colunas, uma com valor minimo e outra com valor maximo
df[['Salary_Min', 'Salary_Max']] = df['Salary_Range'].str.split('-', expand=True)

In [9]:
#Iremos remover o sinal '$'das colunas novas para trasnformar os valores no tipo numerico, assim poderemos realizar operações matematicas com eles.	
df['Salary_Min'] = df['Salary_Min'].str.replace('$', '')
df['Salary_Max'] = df['Salary_Max'].str.replace('$', '')

In [10]:
df.head(5)

,Job_Listing,Job Control,Salary_Range,Work_Type/Schedule,Department,Location,Publish_Date,Filing_Deadline,URLs,Salary_Min,Salary_Max
Working_Title,,,,,,,,,,,
ACCOUNTANT TRAINEE,ACCOUNTANT TRAINEE,403508,$4065.00 - $4847.00,Permanent Fulltime,State Controller's Office,Sacramento County,2023-11-17,2023-11-27,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,4065.00,4847.00
Payroll Disbursement Staff,ACCOUNTING OFFICER (SPECIALIST),397289,$4819.00 - $6032.00,Permanent Fulltime,Department of Water Resources,Sacramento County,2023-11-17,2023-11-27,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,4819.00,6032.00
INFORMATION TECHNOLOGY MANAGER I,INFORMATION TECHNOLOGY MANAGER I,403219,$8591.00 - $11512.00,Permanent Fulltime,Department of Water Resources,Sacramento County,2023-11-17,2023-12-18,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,8591.00,11512.00
CSC Support,INFORMATION TECHNOLOGY SPECIALIST I,403517,$5960.00 - $9643.00,Permanent Fulltime,State Controller's Office,Sacramento County,2023-11-17,NaT,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,5960.00,9643.00
Property Appraiser,JUNIOR PROPERTY APPRAISER,403535,$3534.00 - $4216.00,Permanent Fulltime,Board of Equalization,Sacramento County,2023-11-17,2023-12-11,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,3534.00,4216.00


In [11]:
#atualizando a tabela no banco de dados
df.to_sql('tabela', con = conn, if_exists = 'replace')

9827

In [12]:
#Iremps avaliar a quantidade de valores nulos por coluna
df.isnull().sum() 

Job_Listing              0
Job Control              0
Salary_Range             0
Work_Type/Schedule       0
Department               0
Location                 0
Publish_Date             0
Filing_Deadline       1697
URLs                     0
Salary_Min               0
Salary_Max               0
dtype: int64

### Análise Exploratória de Dados

Após a conclusão do processo de limpeza dos dados, avançamos para as análises exploratórias. O objetivo principal é compreender melhor nossos dados e identificar informações relevantes para orientar a tomada de decisões. Nesta fase, a pergunta central que deve guiar nossos esforços é: "Como posso extrair valor significativo a partir dos dados disponíveis?"

Ao manter o foco na geração de valor, estaremos direcionando nossos esforços para insights práticos e aplicáveis.

A análise exploratória desempenha um papel fundamental no processo de extração de conhecimento dos dados, contribuindo para decisões informadas e estratégias mais eficazes.


In [13]:
query = '''
SELECT *
FROm tabela
LIMIT 5
'''
df_read = pd.read_sql(query, con = conn)
df_read

,Working_Title,Job_Listing,Job Control,Salary_Range,Work_Type/Schedule,Department,Location,Publish_Date,Filing_Deadline,URLs,Salary_Min,Salary_Max
0,ACCOUNTANT TRAINEE,ACCOUNTANT TRAINEE,403508,$4065.00 - $4847.00,Permanent Fulltime,State Controller's Office,Sacramento County,2023-11-17T00:00:00,2023-11-27T00:00:00,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,4065.00,4847.00
1,Payroll Disbursement Staff,ACCOUNTING OFFICER (SPECIALIST),397289,$4819.00 - $6032.00,Permanent Fulltime,Department of Water Resources,Sacramento County,2023-11-17T00:00:00,2023-11-27T00:00:00,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,4819.00,6032.00
2,INFORMATION TECHNOLOGY MANAGER I,INFORMATION TECHNOLOGY MANAGER I,403219,$8591.00 - $11512.00,Permanent Fulltime,Department of Water Resources,Sacramento County,2023-11-17T00:00:00,2023-12-18T00:00:00,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,8591.00,11512.00
3,CSC Support,INFORMATION TECHNOLOGY SPECIALIST I,403517,$5960.00 - $9643.00,Permanent Fulltime,State Controller's Office,Sacramento County,2023-11-17T00:00:00,None,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,5960.00,9643.00
4,Property Appraiser,JUNIOR PROPERTY APPRAISER,403535,$3534.00 - $4216.00,Permanent Fulltime,Board of Equalization,Sacramento County,2023-11-17T00:00:00,2023-12-11T00:00:00,https://www.calcareers.ca.gov/CalHrPublic/Jobs...,3534.00,4216.00


In [35]:
#Qual o top 10 departamentos com mais vagas abertas?
query = '''
SELECT Department, COUNT(*) AS Quantidade
FROM tabela
WHERE Filing_Deadline is null
GROUP BY Department
ORDER BY Quantidade DESC
Limit 10
'''
df_read = pd.read_sql(query, con = conn)
df_read

,Department,Quantidade
0,California Correctional Health Care Services,690
1,Department of State Hospitals,158
2,Department of Veterans Affairs,147
3,Department of Food & Agriculture,93
4,Department of Developmental Services - Porterv...,51
5,Department of Developmental Services - Headqua...,48
6,Department of Transportation,42
7,Department of Rehabilitation,41
8,Department of Social Services,38
9,Department of Justice,28


In [15]:
#Qual o percentual de vagas por departamento?
query = '''
SELECT Department, ROUND(COUNT(*) * 100 / (SELECT COUNT(*) FROM tabela), 2) AS Percentual
FROM tabela
GROUP BY Department
ORDER BY Percentual DESC
LIMIT 15
'''
df_read = pd.read_sql(query, con = conn)
df_read

,Department,Percentual
0,California Correctional Health Care Services,19.0
1,Department of Transportation,8.0
2,Department of State Hospitals,3.0
3,Department of Public Health,3.0
4,Department of Water Resources,2.0
5,Department of Veterans Affairs,2.0
6,Department of Social Services,2.0
7,Department of Justice,2.0
8,Department of General Services,2.0
9,Department of Forestry & Fire Protection,2.0


In [16]:
#Quais vagas são mais comuns?
query = '''
SELECT Job_Listing, COUNT(*) AS Quantidade
FROM tabela
GROUP BY Job_Listing
ORDER BY Quantidade DESC
LIMIT 10
'''
df_read = pd.read_sql(query, con = conn)
df_read

,Job_Listing,Quantidade
0,ASSOCIATE GOVERNMENTAL PROGRAM ANALYST,1443
1,CLINICAL SOCIAL WORKER (HEALTH/CORRECTIONAL FA...,288
2,INFORMATION TECHNOLOGY SPECIALIST I,175
3,CERTIFIED NURSING ASSISTANT,174
4,OFFICE TECHNICIAN (TYPING),161
5,"PSYCHOLOGIST - CLINICAL, CORRECTIONAL FACILITY",157
6,STAFF SERVICES ANALYST,153
7,STAFF SERVICES MANAGER I,142
8,CALTRANS EQUIPMENT OPERATOR II,141
9,C. E. A.,141


In [17]:
#Quais vagas são mais comuns dentro do deaprtamento 'California Correctional Health Care Services'?
query = '''
SELECT Job_Listing, COUNT(*) AS Quantidade
FROM tabela
WHERE Department = 'California Correctional Health Care Services'
GROUP BY Job_Listing
ORDER BY Quantidade DESC
LIMIT 10
'''
df_read = pd.read_sql(query, con = conn)
df_read

,Job_Listing,Quantidade
0,CLINICAL SOCIAL WORKER (HEALTH/CORRECTIONAL FA...,272
1,"PSYCHOLOGIST - CLINICAL, CORRECTIONAL FACILITY",157
2,CERTIFIED NURSING ASSISTANT,139
3,"PHYSICIAN AND SURGEON, CORRECTIONAL FACILITY (...",89
4,"REGISTERED NURSE, CORRECTIONAL FACILITY",86
5,CORRECTIONAL SUPERVISING COOK (CORRECTIONAL FA...,85
6,"LICENSED VOCATIONAL NURSE, CALIFORNIA DEPARTME...",74
7,"STAFF PSYCHIATRIST, CORRECTIONAL AND REHAB SER...",70
8,MEDICAL ASSISTANT,63
9,ASSOCIATE GOVERNMENTAL PROGRAM ANALYST,59


In [18]:
#Quais os dados estatisticos dos salarios minimos ?
query = '''
SELECT
    MIN(Salary_Min) AS Minimo,
    ROUND(AVG(Salary_Min), 2) AS Media,
    MAX(Salary_Min) AS Maximo
FROM tabela
'''
df_read = pd.read_sql(query, con = conn)
df_read

,Minimo,Media,Maximo
0,0.00,5876.23,9981.00


In [19]:
#Qual é o percentil 75 dos salarios minimos?
query = '''
SELECT
    Salary_Min,
    percentil
FROM (SELECT ntile(100) OVER (ORDER BY Salary_Min) AS percentil, Salary_Min FROM tabela) as a
WHERE percentil = 75
LIMIT 1
'''
df_read = pd.read_sql(query, con = conn)
df_read

,Salary_Min,percentil
0,6061.00,75


In [26]:
#Qual a difereça média entre o salário mínimo e o salário máximo por vaga?
query = '''
SELECT Distinct
    Department,
    Job_Listing,
    ROUND(AVG(Salary_Max - Salary_Min), 2) AS Diferenca
FROM tabela
GROUP BY 1,2
ORDER BY Diferenca DESC
LIMIT 10
'''
df_read = pd.read_sql(query, con = conn)
df_read

,Department,Job_Listing,Diferenca
0,Public Employees Retirement System,"CHIEF INVESTMENT OFFICER, PUBLIC EMPLOYEES RET...",23583.33
1,Public Employees Retirement System,"INVESTMENT DIRECTOR, PUBLIC EMPLOYEES' RETIREM...",13375.00
2,Governor's Office of Planning and Research,"ASSISTANT TO THE GOVERNOR, OFFICE OF PLANNING ...",12528.00
3,California Correctional Institution,VOCATIONAL INSTRUCTOR (COMPUTER AND RELATED TE...,11717.67
4,Correctional Training Facility,TEACHER (HIGH SCHOOL-PHYSICAL EDUCATION) (CORR...,11717.67
5,Ironwood State Prison,VOCATIONAL INSTRUCTOR (CARPENTRY) (CORRECTIONA...,11715.63
6,Ironwood State Prison,VOCATIONAL INSTRUCTOR (WELDING) (CORRECTIONAL ...,11715.63
7,Ironwood State Prison,"VOCATIONAL INSTRUCTOR, AUTO MECHANICS (CORRECT...",11715.63
8,Mule Creek State Prison,CORRECTIONAL COUNSELOR II (SPECIALIST),11569.97
9,"California State Prison, Solano",CORRECTIONAL LIEUTENANT,11198.47


In [32]:
#Qual a média de tempo que uma vaga fica aberta por departamento?
query = '''
SELECT
    Department,
    ROUND(AVG(julianday(Filing_Deadline) - julianday(Publish_Date)), 2) AS Media
FROM tabela
GROUP BY Department
ORDER BY Media DESC
LIMIT 10
'''
df_read = pd.read_sql(query, con = conn)
df_read

,Department,Media
0,Department of State Hospitals - Atascadero,79.75
1,Commission on Ca State Govt Organization & Eco...,63.00
2,State Personnel Board,62.00
3,Department of State Hospitals - Coalinga,44.00
4,Administrative Law Office,38.00
5,Delta Protection Commission,31.50
6,Agricultural Labor Relations Board,30.00
7,California Men's Colony,29.81
8,Department of Parks & Recreation,28.98
9,Department of Consumer Affairs,28.26


In [34]:
#Quais vagas são finalizados mais rapidamente?
query = '''
SELECT
    Job_Listing,
    ROUND(AVG(julianday(Filing_Deadline) - julianday(Publish_Date)), 2) AS Media
FROM tabela
WHERE Filing_Deadline IS NOT NULL
GROUP BY Job_Listing
ORDER BY Media 
LIMIT 10
'''
df_read = pd.read_sql(query, con = conn)
df_read

,Job_Listing,Media
0,ACCOUNT CLERK II,10.0
1,ACCOUNTING ADMINISTRATOR III,10.0
2,"ACTIVITY COORDINATOR, VETERANS HOME AND MEDICA...",10.0
3,"ADMINISTRATOR, SCIENCE PROGRAM, CALIFORNIA SCI...",10.0
4,ASSISTANT STATE ARCHEOLOGIST,10.0
5,ASSOCIATE MECHANICAL ENGINEER,10.0
6,ASSOCIATE TRANSPORTATION OPERATIONS SUPERVISOR...,10.0
7,AUDIO-VISUAL SPECIALIST (TECHNICAL),10.0
8,BUSINESS SYSTEMS ANALYST,10.0
9,CHIEF HYDROELECTRIC PLANT OPERATOR,10.0
